<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/train_undefined_presence_ai_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
train_undefined_presence_ai.py

1. Synthetic dataset (6→3)
2. Float32 normalization
3. MC-Dropout model
4. Physics-informed residual loss
5. AdamW training with scheduler, clipping, early stop
6. MC-Dropout inference mean/std
7. UMAP latent visualization
"""

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt
import umap
import random

# 1. Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. Synthetic Dataset
class UndefinedPresenceDataset(Dataset):
    def __init__(self, n=5000):
        np.random.seed(SEED)
        u = np.random.uniform(-1,1,(n,1))
        v = np.random.uniform(0,2,(n,1))
        w = np.random.uniform(-2,2,(n,1))
        x = np.random.uniform(0,5,(n,1))
        y = np.random.uniform(-1,1,(n,1))
        z = np.random.uniform(0,1,(n,1))
        X = np.hstack([u,v,w,x,y,z]).astype(np.float32)

        # toy physics:
        # presence = sin(u)*v + cos(w)
        # dissolution = exp(-x*y)
        # transcendence = z * (presence + dissolution)
        presence = np.sin(u)*v + np.cos(w)
        dissolution = np.exp(-x*y)
        transcendence = z*(presence+dissolution)
        Y = np.hstack([presence, dissolution, transcendence]).astype(np.float32)
        Y += 0.01 * Y.std(axis=0) * np.random.randn(*Y.shape).astype(np.float32)

        self.X_mean, self.X_std = X.mean(0), X.std(0)+1e-8
        self.Y_mean, self.Y_std = Y.mean(0), Y.std(0)+1e-8
        self.X = ((X-self.X_mean)/self.X_std).astype(np.float32)
        self.Y = ((Y-self.Y_mean)/self.Y_std).astype(np.float32)

    def __len__(self):
        return len(self.X)
    def __getitem__(self,i):
        return torch.from_numpy(self.X[i]), torch.from_numpy(self.Y[i])

# 3. Model with MC-Dropout
class UndefinedPresenceAI(nn.Module):
    def __init__(self, inp=6, hid=32, out=3, p=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(inp,hid),
            nn.ReLU(),
            nn.Dropout(p),
            nn.Linear(hid,out)
        )
    def forward(self,x):
        return self.net(x)

# 4. Physics-Informed Loss
def physics_residual(pred, X, stats):
    X_den = X*stats['X_std']+stats['X_mean']
    u,v,w,x,y,z = X_den.T
    presence = torch.sin(u)*v + torch.cos(w)
    dissolution = torch.exp(-x*y)
    transcend = z*(presence+dissolution)
    Yt = torch.stack([presence,dissolution,transcend],1)
    Yn = (Yt-stats['Y_mean'])/stats['Y_std']
    return nn.MSELoss()(pred,Yn)

def total_loss(pred, true, X, stats, lam=1.0):
    l = nn.MSELoss()(pred,true)
    p = physics_residual(pred,X,stats)
    return l+lam*p, l, p

# 5. MC-Dropout Prediction
def mc_predict(model, X, T=50):
    model.train()
    ps=[]
    with torch.no_grad():
        for _ in range(T):
            ps.append(model(X))
    arr=torch.stack(ps)
    return arr.mean(0), arr.std(0)

# 6. Training Loop
def train(model, dl_tr, dl_va, stats, epochs=100, lr=1e-3, wd=1e-5, lam=1.0, patience=10):
    model.to(DEVICE)
    opt=optim.AdamW(model.parameters(),lr=lr,weight_decay=wd)
    sch=optim.lr_scheduler.ReduceLROnPlateau(opt,'min',factor=0.5,patience=5)
    best=1e9; wait=0; hist={'tr':[],'va':[]}
    for e in range(1,epochs+1):
        model.train(); tr=0
        for Xb,Yb in dl_tr:
            Xb,Yb=Xb.to(DEVICE),Yb.to(DEVICE)
            pred=model(Xb)
            loss,_,_=total_loss(pred,Yb,Xb,stats,lam)
            opt.zero_grad();loss.backward();opt.step()
            tr+=loss.item()*len(Xb)
        tr/=len(dl_tr.dataset)
        model.eval(); va=0
        with torch.no_grad():
            for Xv,Yv in dl_va:
                Xv,Yv=Xv.to(DEVICE),Yv.to(DEVICE)
                pred=model(Xv)
                loss,_,_=total_loss(pred,Yv,Xv,stats,lam)
                va+=loss.item()*len(Xv)
        va/=len(dl_va.dataset)
        sch.step(va); hist['tr'].append(tr); hist['va'].append(va)
        print(f"Epoch {e} | Train {tr:.4f} | Val {va:.4f}")
        if va<best-1e-6: best=va; wait=0;torch.save(model.state_dict(),"best.pth")
        else: wait+=1
        if wait>=patience: break
    model.load_state_dict(torch.load("best.pth")); return hist,model

# 7. Main Execution
if __name__=="__main__":
    ds=UndefinedPresenceDataset()
    nva=int(0.2*len(ds))
    tr,va=random_split(ds,[len(ds)-nva,nva])
    dlt=DataLoader(tr,batch_size=128,shuffle=True)
    dlv=DataLoader(va,batch_size=256)
    stats={k:torch.tensor(v,device=DEVICE) for k,v in ds.__dict__.items() if k.endswith(('mean','std'))}
    m=UndefinedPresenceAI().to(DEVICE)
    history,model=train(m,dlt,dlv,stats)
    # inference
    Xall=torch.from_numpy(ds.X).to(DEVICE)
    mean,std=mc_predict(model,Xall)
    print("Mean:",mean[0].cpu().numpy(), "Std:",std[0].cpu().numpy())
    # UMAP
    feats=model.net[0](Xall).detach().cpu().numpy()
    emb=umap.UMAP(n_components=2,random_state=SEED).fit_transform(feats)
    plt.scatter(emb[:,0],emb[:,1],s=4,alpha=0.5);plt.show()